In [4]:
! wget https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt -O words.txt

--2022-02-07 23:46:36--  https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234901 (4.0M) [text/plain]
Saving to: ‘words.txt’

words.txt           100%[===================>]   4.04M  14.5MB/s    in 0.3s    

2022-02-07 23:46:36 (14.5 MB/s) - ‘words.txt’ saved [4234901/4234901]



In [93]:
# Read the dictionary into a list.
with open('words.txt') as file:
    ALL_WORDS = []
    for line in file:
        ALL_WORDS.append(line.rstrip())

print(f'Word #1237 is "{ALL_WORDS[1237]}".')

Word #1237 is "academised".


In [94]:
def filter_by_length(words, length):
    return list(filter(lambda x: len(x) == length, words))

for i in range(1,15):
    print(f'{i}-letter words: {len(filter_by_length(ALL_WORDS, i))}')

1-letter words: 26
2-letter words: 427
3-letter words: 2130
4-letter words: 7186
5-letter words: 15918
6-letter words: 29874
7-letter words: 41998
8-letter words: 51627
9-letter words: 53402
10-letter words: 45872
11-letter words: 37539
12-letter words: 29125
13-letter words: 20944
14-letter words: 14149


In [102]:
def filter_by_minimum(words, letter, minimum=1):
    return list(filter(lambda x: x.count(letter) >= minimum, words))

def filter_by_maximum(words, letter, maximum):
    return list(filter(lambda x: x.count(letter) <= maximum, words))

print(f'Words with at least 4 z\'s in them: {filter_by_minimum(ALL_WORDS, "z", minimum=4)}.')

print(f'Words with at least 4 z\'s in them AND no p\'s: {filter_by_maximum(filter_by_minimum(ALL_WORDS, "z", minimum=4), "p", maximum=0)}.')

Words with at least 4 z's in them: ['pizzazz', 'pizzazzes', 'razzmatazz'].
Words with at least 4 z's in them AND no p's: ['razzmatazz'].


In [96]:
def filter_by_position(words, letter, position):
    return list(filter(lambda x: position < len(x) and x[position] == letter, words))

print(f'Words with an x in the 17th place: {filter_by_position(ALL_WORDS, "x", 16)}')

Words with an x in the 17th place: ['hydropneumothorax', 'pneumohydrothorax']


In [98]:
import random
import string

class Wordle:

    def __init__(self, length=5):
        self.word = random.choice(filter_by_length(ALL_WORDS, length))

    def __get_letter_count(self):
        return {x: self.word.count(x) for x in string.ascii_lowercase}

    def guess(self, guess):
        assert(len(guess) == len(self.word))
        letter_count = self.__get_letter_count()
        result = [None] * len(self.word)
        # Add correct guesses to result first.
        for i, letter in enumerate(guess):
            if self.word[i] == letter:
                result[i] = (0, letter)
                letter_count[letter] -= 1
        ## Add incorrect guesses to result.
        for i, letter in enumerate(guess):
            if result[i] == None:
                if letter_count[letter] > 0:
                    result[i] = (1, letter)
                else:
                    result[i] = (2, letter)
                letter_count[letter] -= 1
        return result


In [187]:
from termcolor import colored

def pretty_guess_result(guess_result):
    colors = {0: 'green', 1: 'yellow', 2: 'red'}
    result = ''
    for guess in guess_result:
        result += colored(guess[1], colors[guess[0]])
    return result

game = Wordle()

print(pretty_guess_result(game.guess('skier')))
print(pretty_guess_result(game.guess('watch')))
print(pretty_guess_result(game.guess('bumpy')))
print(pretty_guess_result(game.guess('lodge')))


skier
watch
bumpy
lodge


In [222]:
class Agent:

    def __init__(self, length=5):
        self.words_remaining = filter_by_length(ALL_WORDS, length)

    def guess(self):
        return random.choice(self.words_remaining)
    
    def update_with_guess_result(self, guess_result):
        min_count = {}
        max_count = {}
        for i, letter_guess in enumerate(guess_result):
            score = letter_guess[0]
            letter = letter_guess[1]
            # Filter correct letters
            if score == 0:
                # print(f'...filtering words without "{letter}" in position {i}')
                self.words_remaining = filter_by_position(self.words_remaining, letter, i)
            # Count correct and partially correct letters
            if score <= 1:
                min_count[letter] = min_count.get(letter, 0) + 1
                if letter in max_count:
                    max_count[letter] = min_count[letter]
            elif score == 2:
                max_count[letter] = min_count.get(letter, 0)
        for letter in min_count.keys():
            # print(f'...filtering by minimum {min_count[letter]} of "{letter}"')
            self.words_remaining = filter_by_minimum(self.words_remaining, letter, min_count[letter])
        for letter in max_count.keys():
            # print(f'...filtering by maximum {max_count[letter]} of "{letter}"')
            self.words_remaining = filter_by_maximum(self.words_remaining, letter, max_count[letter])
        

def play_game_simple(length=5):
    game = Wordle(length=length)
    agent = Agent(length=length)

    print(f'Starting a game with a {length}-letter word.')
    while True:
        word = agent.guess()
        result = game.guess(word)
        agent.update_with_guess_result(result)
        print(pretty_guess_result(result))
        if word == game.word:
            break


    print(f'The word was: "{game.word}".')

def play_game_manual(length=5):
    agent = Agent(length=length)

    print(f'Starting a game with a {length}-letter word.')
    while True:
        word = agent.guess()
        print(word)
        input('...')
        score = input('Enter the score: ')
        if score.isnumeric():
            guess_result = []
            for i, letter in enumerate(word):
                guess_result.append((int(score[i]), letter))
            agent.update_with_guess_result(guess_result)
            print(pretty_guess_result(guess_result))


play_game_simple()
# play_game_manual(length=10)

Starting a game with a 5-letter word.
ifrit
bulge
kassu
kudzu
kumyk
kokum
The word was: "kokum".
